# Create Payment Drafts with Revolut Business API

This notebook shows how to create payment drafts using the Revolut Business API. Payment drafts appear in your customer's Revolut Business app for review and approval before execution.

**Workflow:**
1. API creates payment draft
2. Customer reviews in Revolut Business app (Transfers → Draft)
3. Customer approves → payment executes

This guide uses the **Sandbox environment** for testing. Switch to production URLs when ready.

## Prerequisites

Your customer needs to:
1. Have a Revolut Business account (Grow plan or above for API access)
2. Go to Settings → APIs → Business API
3. Upload your public certificate and set OAuth redirect URI
4. Share the `client_id` with you
5. Grant consent by clicking "Enable access"

## References

- [Business API docs](https://developer.revolut.com/docs/business/business-api)
- [Payment drafts guide](https://developer.revolut.com/docs/guides/manage-accounts/transfers/payment-drafts)
- [Sandbox login](https://sandbox-business.revolut.com/signin)

## Setup

In [ ]:
import os
import time
import uuid
import requests
import jwt
from dotenv import load_dotenv

load_dotenv()

# Environment: 'sandbox' or 'production'
ENVIRONMENT = 'sandbox'

# API base URLs
BASE_URLS = {
    'sandbox': 'https://sandbox-b2b.revolut.com/api/1.0',
    'production': 'https://b2b.revolut.com/api/1.0'
}

AUTH_URLS = {
    'sandbox': 'https://sandbox-b2b.revolut.com/api/1.0/auth/token',
    'production': 'https://b2b.revolut.com/api/1.0/auth/token'
}

BASE_URL = BASE_URLS[ENVIRONMENT]
AUTH_URL = AUTH_URLS[ENVIRONMENT]

## Configuration

Create a `.env` file with:
```
REVOLUT_CLIENT_ID=<your_client_id>
REVOLUT_REFRESH_TOKEN=<your_refresh_token>
REVOLUT_REDIRECT_DOMAIN=example.com
```

And ensure `private_key.pem` exists in the same directory (the private key matching the public cert uploaded to Revolut).

In [ ]:
# Load credentials
CLIENT_ID = os.getenv('REVOLUT_CLIENT_ID')
REFRESH_TOKEN = os.getenv('REVOLUT_REFRESH_TOKEN')  # Obtained during initial OAuth consent
REDIRECT_DOMAIN = os.getenv('REVOLUT_REDIRECT_DOMAIN', 'example.com')  # Without https://

# Load private key for JWT signing
with open('privatecert.pem', 'r') as f:
    PRIVATE_KEY = f.read()

print(f"Environment: {ENVIRONMENT}")
print(f"Client ID: {CLIENT_ID[:20]}..." if CLIENT_ID else "CLIENT_ID not set!")
print(f"Refresh token: {'Set' if REFRESH_TOKEN else 'Not set - need initial OAuth flow'}")

## Authentication

Revolut uses OAuth2 with JWT client assertions. Access tokens expire every 40 minutes.

In [ ]:
def generate_client_assertion(client_id: str, private_key: str, redirect_domain: str) -> str:
    """
    Generate a JWT client assertion for Revolut API authentication.
    
    The JWT is signed with your private key (matching the public cert uploaded to Revolut).
    """
    payload = {
        'iss': redirect_domain,  # Must match OAuth redirect URI domain (without https://)
        'sub': client_id,
        'aud': 'https://revolut.com',
        'exp': int(time.time()) + 3600  # 1 hour expiry
    }
    
    return jwt.encode(payload, private_key, algorithm='RS256')


def refresh_access_token(refresh_token: str, client_assertion: str) -> dict:
    """
    Exchange refresh token for a new access token.
    
    Access tokens are valid for 40 minutes.
    Refresh tokens don't expire (except for Freelancer plans: 90 days).
    """
    response = requests.post(
        AUTH_URL,
        headers={'Content-Type': 'application/x-www-form-urlencoded'},
        data={
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer',
            'client_assertion': client_assertion
        }
    )
    response.raise_for_status()
    return response.json()


# Get access token
client_assertion = generate_client_assertion(CLIENT_ID, PRIVATE_KEY, REDIRECT_DOMAIN)
token_response = refresh_access_token(REFRESH_TOKEN, client_assertion)

ACCESS_TOKEN = token_response['access_token']
print(f"Access token obtained, expires in {token_response['expires_in']} seconds")

In [ ]:
# Helper for authenticated requests
def api_request(method: str, endpoint: str, **kwargs) -> requests.Response:
    """Make an authenticated request to the Revolut Business API."""
    headers = kwargs.pop('headers', {})
    headers['Authorization'] = f'Bearer {ACCESS_TOKEN}'
    
    url = f"{BASE_URL}{endpoint}"
    response = requests.request(method, url, headers=headers, **kwargs)
    
    if not response.ok:
        print(f"Error {response.status_code}: {response.text}")
    
    return response

## Step 1: Get Accounts

Find the account ID to pay from.

In [ ]:
def get_accounts() -> list:
    """Get all accounts for this business."""
    response = api_request('GET', '/accounts')
    response.raise_for_status()
    return response.json()


accounts = get_accounts()
print(f"Found {len(accounts)} account(s):\n")

for acc in accounts:
    print(f"  {acc['currency']} - {acc.get('name', '(unnamed)')}")
    print(f"    ID: {acc['id']}")
    print(f"    Balance: {acc['balance']} {acc['currency']}")
    print(f"    State: {acc['state']}")
    print()

In [ ]:
# Select the account to pay from (e.g., GBP account)
# In production, you'd select based on the payment currency
SOURCE_ACCOUNT_ID = accounts[0]['id']  # Adjust as needed
print(f"Using source account: {SOURCE_ACCOUNT_ID}")

## Step 2: Create Counterparty (Recipient)

Unlike Wise, Revolut requires you to pre-register recipients as "counterparties" before you can pay them.

**Sandbox note:** In sandbox, you can only add test Revolut users via Revtag:
- `Test User 1` / `john1pvki`
- `Test User 2` / `john2pvki`
- ... through `Test User 9` / `john9pvki`

In [ ]:
def create_counterparty_revtag(name: str, revtag: str, profile_type: str = 'personal') -> dict:
    """
    Create a counterparty using their Revolut Revtag.
    
    This is the easiest way to add another Revolut user.
    For sandbox testing, use the test accounts.
    """
    response = api_request(
        'POST', '/counterparty',
        json={
            'name': name,
            'revtag': revtag,
            'profile_type': profile_type
        }
    )
    response.raise_for_status()
    return response.json()


def create_counterparty_bank_account(
    name: str,
    currency: str,
    bank_country: str,
    account_no: str = None,
    sort_code: str = None,
    iban: str = None,
    bic: str = None,
    address: dict = None
) -> dict:
    """
    Create a counterparty using bank account details.
    
    For UK (GBP): use account_no + sort_code
    For SEPA (EUR): use iban
    For SWIFT: use iban + bic
    """
    payload = {
        'company_name': name,  # Use 'individual_name' for persons
        'currency': currency,
        'bank_country': bank_country,
    }
    
    if account_no:
        payload['account_no'] = account_no
    if sort_code:
        payload['sort_code'] = sort_code
    if iban:
        payload['iban'] = iban
    if bic:
        payload['bic'] = bic
    if address:
        payload['address'] = address
    
    response = api_request('POST', '/counterparty', json=payload)
    response.raise_for_status()
    return response.json()

In [ ]:
# Create a test counterparty (sandbox)
counterparty = create_counterparty_revtag(
    name='Test User 2',
    revtag='john2pvki',
    profile_type='personal'
)

print(f"Created counterparty:")
print(f"  ID: {counterparty['id']}")
print(f"  Name: {counterparty['name']}")
print(f"  State: {counterparty['state']}")

COUNTERPARTY_ID = counterparty['id']

In [ ]:
# Alternative: List existing counterparties
def get_counterparties() -> list:
    """Get all counterparties."""
    response = api_request('GET', '/counterparties')
    response.raise_for_status()
    return response.json()


counterparties = get_counterparties()
print(f"Found {len(counterparties)} counterparty(ies)")
for cp in counterparties[:5]:  # Show first 5
    print(f"  {cp['name']} - ID: {cp['id']}")

In [ ]:
COUNTERPARTY_ID = "15c1155f-0992-4157-8d75-4883f1730385"

## Step 3: Create Payment Draft

This is the core functionality. The draft will appear in the customer's Revolut Business app for review.

In [ ]:
def create_payment_draft(
    account_id: str,
    counterparty_id: str,
    amount: float,
    currency: str,
    reference: str,
    title: str = None,
    schedule_for: str = None,  # ISO date, e.g., '2025-01-15'
    counterparty_account_id: str = None  # If counterparty has multiple accounts
) -> dict:
    """
    Create a payment draft.
    
    The draft stays pending until the account owner reviews and approves it
    in the Revolut Business app (Transfers → Draft → Send).
    
    Args:
        account_id: Your account to pay from
        counterparty_id: The recipient (must be pre-created)
        amount: Payment amount
        currency: 3-letter currency code (e.g., 'GBP', 'EUR')
        reference: Payment reference (appears on recipient's statement)
        title: Optional title for the draft (for organization)
        schedule_for: Optional future date for scheduled payment
        counterparty_account_id: Specify if counterparty has multiple accounts
    """
    receiver = {'counterparty_id': counterparty_id}
    if counterparty_account_id:
        receiver['account_id'] = counterparty_account_id
    
    payload = {
        'payments': [{
            'account_id': account_id,
            'receiver': receiver,
            'amount': amount,
            'currency': currency,
            'reference': reference
        }]
    }
    
    if title:
        payload['title'] = title
    if schedule_for:
        payload['schedule_for'] = schedule_for
    
    response = api_request('POST', '/payment-drafts', json=payload)
    response.raise_for_status()
    return response.json()

In [ ]:
# Create a payment draft!
draft = create_payment_draft(
    account_id=SOURCE_ACCOUNT_ID,
    counterparty_id=COUNTERPARTY_ID,
    amount=700.00,
    currency='GBP',
    reference='Demo payment for Jamie from Reimagine Robotics',
    title='API Test Draft'
)

print(f"✓ Payment draft created!")
print(f"  Draft ID: {draft['id']}")
print(f"")
print(f"Next step: Customer reviews in Revolut Business app")
print(f"  → Transfers → Draft → Select draft → Send")

## Bulk Payments

You can include multiple payments in a single draft. All payments must use the same source account.

In [ ]:
def create_bulk_payment_draft(
    account_id: str,
    payments: list,  # List of {counterparty_id, amount, currency, reference}
    title: str = None
) -> dict:
    """
    Create a bulk payment draft with multiple payments.
    
    All payments must be from the same source account.
    """
    payload = {
        'payments': [
            {
                'account_id': account_id,
                'receiver': {'counterparty_id': p['counterparty_id']},
                'amount': p['amount'],
                'currency': p['currency'],
                'reference': p['reference']
            }
            for p in payments
        ]
    }
    
    if title:
        payload['title'] = title
    
    response = api_request('POST', '/payment-drafts', json=payload)
    response.raise_for_status()
    return response.json()


# Example bulk payment (commented out - uncomment to test)
# bulk_draft = create_bulk_payment_draft(
#     account_id=SOURCE_ACCOUNT_ID,
#     payments=[
#         {'counterparty_id': COUNTERPARTY_ID, 'amount': 5.00, 'currency': 'GBP', 'reference': 'Payment 1'},
#         {'counterparty_id': COUNTERPARTY_ID, 'amount': 7.50, 'currency': 'GBP', 'reference': 'Payment 2'},
#     ],
#     title='Bulk API Test'
# )
# print(f"Bulk draft created: {bulk_draft['id']}")

## Manage Drafts

List, retrieve, and delete payment drafts.

In [ ]:
def get_payment_drafts() -> list:
    """Get all payment drafts created via API (not processed yet)."""
    response = api_request('GET', '/payment-drafts')
    response.raise_for_status()
    return response.json()


def get_payment_draft(draft_id: str) -> dict:
    """Get details of a specific payment draft."""
    response = api_request('GET', f'/payment-drafts/{draft_id}')
    response.raise_for_status()
    return response.json()


def delete_payment_draft(draft_id: str) -> None:
    """Delete a payment draft (only if not yet sent for processing)."""
    response = api_request('DELETE', f'/payment-drafts/{draft_id}')
    response.raise_for_status()
    print(f"Draft {draft_id} deleted")


# List all drafts
drafts = get_payment_drafts()
print(f"Found {len(drafts)} draft(s):\n")

for d in drafts:
    print(f"  Draft ID: {d['id']}")
    if 'title' in d:
        print(f"    Title: {d['title']}")
    print()

## Initial OAuth Setup (First Time Only)

If you don't have a refresh token yet, you need to complete the initial OAuth consent flow.

This is a one-time setup performed by the customer in their Revolut Business app.

In [ ]:
def get_authorization_url(client_id: str, redirect_uri: str, scopes: list = None) -> str:
    """
    Generate the OAuth authorization URL.
    
    The customer clicks this URL, logs in, and grants consent.
    They'll be redirected to redirect_uri with a ?code= parameter.
    
    Scopes:
    - READ: View accounts, transactions, counterparties
    - WRITE: Create/update counterparties, webhooks, payment drafts
    - PAY: Execute payments (Company plans only)
    
    For payment drafts, you only need READ and WRITE.
    """
    base = 'https://sandbox-business.revolut.com' if ENVIRONMENT == 'sandbox' else 'https://business.revolut.com'
    
    url = f"{base}/app-confirm?client_id={client_id}&redirect_uri={redirect_uri}&response_type=code"
    
    if scopes:
        url += f"&scope={','.join(scopes)}"
    
    return url


def exchange_code_for_tokens(authorization_code: str, client_assertion: str) -> dict:
    """
    Exchange the authorization code for access and refresh tokens.
    
    The authorization code is only valid for 2 minutes!
    """
    response = requests.post(
        AUTH_URL,
        headers={'Content-Type': 'application/x-www-form-urlencoded'},
        data={
            'grant_type': 'authorization_code',
            'code': authorization_code,
            'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer',
            'client_assertion': client_assertion
        }
    )
    response.raise_for_status()
    return response.json()


# Generate authorization URL (share this with customer)
if CLIENT_ID:
    auth_url = get_authorization_url(
        client_id=CLIENT_ID,
        redirect_uri='https://example.com',  # Must match what's configured in Revolut
        scopes=['READ', 'WRITE']  # No PAY needed for drafts
    )
    print("Authorization URL (customer opens this):")
    print(auth_url)

In [ ]:
# After customer authorizes, they'll be redirected to:
# https://example.com?code=oa_sand_xxxxx
#
# Paste that code here and run to get your tokens:

# AUTHORIZATION_CODE = 'oa_sand_xxxxx'  # Paste the code here
#
# client_assertion = generate_client_assertion(CLIENT_ID, PRIVATE_KEY, REDIRECT_DOMAIN)
# tokens = exchange_code_for_tokens(AUTHORIZATION_CODE, client_assertion)
#
# print("Save these tokens!")
# print(f"Access token: {tokens['access_token']}")
# print(f"Refresh token: {tokens['refresh_token']}")
# print(f"\nAdd to .env: REVOLUT_REFRESH_TOKEN={tokens['refresh_token']}")

## Summary: Comparison with Wise

| Aspect | Wise | Revolut |
|--------|------|----------|
| Auth | API token + SCA private key | OAuth2 + JWT client assertion |
| Token lifetime | Long-lived | 40 min access, refresh to renew |
| Recipients | Inline in transfer call | Must pre-create as counterparty |
| Payment initiation | Direct transfer | Payment draft (review in app) |
| Approval | SCA signature | Human review in app |
| Bulk payments | Loop over transfers | Single draft with multiple payments |